# Policy Based Agent: Actor Critic 
### <=> 가치 기반 에이전트: 결정론적 특징, 각 상태와 액션이 가지는 가치가 결국엔 특정한 값으로 수렴
- 정책 에이전트의 경우 환경에서 정채겡 맞춰 움직이는 에이전트가 경험을 쌓게 해서 강화해 나감
- 네트워크 이외에도 손실 함수(목적 함수)를 정의해야 함. 
## Actor Critic: 정책 네트워크와 밸류 네트워크를 함께 학습하는 것
- Q 액터 크리틱
- 어드벤티지 액터 크리틱
- TD 액터 크리틱

In [1]:
# Import Lib
import gym
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

In [2]:
#Hyperparameters
learning_rate = 0.0002
gamma         = 0.98
n_rollout     = 10

In [3]:
class ActorCritic(nn.Module):
    def __init__(self):
        super(ActorCritic, self).__init__()
        self.data = []
        
        self.fc1 = nn.Linear(4,256) 
        self.fc_pi = nn.Linear(256,2)
        self.fc_v = nn.Linear(256,1)
        self.optimizer = optim.Adam(self.parameters(), lr=learning_rate)
        
    def pi(self, x, softmax_dim = 0): # 입력 x에 대한 정책 구하는 함수
        x = F.relu(self.fc1(x))
        x = self.fc_pi(x)
        prob = F.softmax(x, dim=softmax_dim) # softmax를 통해 확률을 구함
        return prob
    
    def v(self, x):  # 입력 x에 대한 가치 함수를 반환하는 함수
        x = F.relu(self.fc1(x)) 
        v = self.fc_v(x)
        return v
    
    def put_data(self, transition): #에피소드의 파라미터들을 리스트에 추가
        self.data.append(transition)
        
    def make_batch(self): #저장된 transition 데이터 배치 형태로 반환하는 함수
        s_lst, a_lst, r_lst, s_prime_lst, done_lst = [], [], [], [], []
        for transition in self.data:
            s,a,r,s_prime,done = transition
            s_lst.append(s)
            a_lst.append([a])
            r_lst.append([r/100.0])
            s_prime_lst.append(s_prime)
            done_mask = 0.0 if done else 1.0
            done_lst.append([done_mask])
        
        s_batch, a_batch, r_batch, s_prime_batch, done_batch = torch.tensor(s_lst, dtype=torch.float), torch.tensor(a_lst), \
                                                               torch.tensor(r_lst, dtype=torch.float), torch.tensor(s_prime_lst, dtype=torch.float), \
                                                               torch.tensor(done_lst, dtype=torch.float)
        self.data = []
        return s_batch, a_batch, r_batch, s_prime_batch, done_batch #최종적으로 텐서로 변환해 반환
  
    def train_net(self): # 네트워크를 학습하는 함수
        s, a, r, s_prime, done = self.make_batch() #저장된 transition 데이터로부터 배치를 만듦
        td_target = r + gamma * self.v(s_prime) * done # 타깃 가치 함수값인 TD 타깃을 계산
        delta = td_target - self.v(s) #: TD 오차인 delta를 계산
        
        pi = self.pi(s, softmax_dim=1) #현재 상태에 대한 정책을 계산
        pi_a = pi.gather(1,a) #현재 상태에서 취한 행동에 대한 확률을 추출
        loss = -torch.log(pi_a) * delta.detach() + F.smooth_l1_loss(self.v(s), td_target.detach()) #: 손실 함수를 계산

        self.optimizer.zero_grad() #옵티마이저의 그래디언트를 초기화
        loss.mean().backward() #손실의 평균을 계산하고, 그래디언트를 역전파
        self.optimizer.step() # 옵티마이저를 사용하여 신경망의 가중치를 업데이트 

In [4]:
def main():  
    env = gym.make('CartPole-v1')
    model = ActorCritic()    
    print_interval = 20
    score = 0.0

    for n_epi in range(1000): 
        done = False
        s, _ = env.reset()
        while not done:
            for t in range(n_rollout):# 에이전트의 액션 선택
                prob = model.pi(torch.from_numpy(s).float()) #현재 상태를 모델에 입력해 정책을 구함
                m = Categorical(prob) #확률 분포를 기반으로 범주형 분포 객체를 생성
                a = m.sample().item() #범주형 분포로부터 행동을 샘플링하여 선택
                s_prime, r, done, truncated, info = env.step(a) #택한 행동을 환경에 적용하여 다음 상태와 보상 등을 얻음
                model.put_data((s,a,r,s_prime,done)) # 데이터를 모델의 버퍼에 저장
                
                s = s_prime # 다음상태를 현재 상태로 업데이트
                score += r #누적보상 계산
                
                if done:
                    break                     
            
            model.train_net() # 모댈 학습
            
        if n_epi%print_interval==0 and n_epi!=0:# 정해진 간격마다 학습결과값 출력
            print("# of episode :{}, avg score : {:.1f}".format(n_epi, score/print_interval))
            score = 0.0 
    env.close() # 환경 닫음

if __name__ == '__main__':
    main()

C:\Users\AIMLAB_JKIM\AppData\Local\Programs\Python\Python311\Lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
C:\Users\AIMLAB_JKIM\AppData\Local\Temp\ipykernel_2932\3860977774.py:36: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:248.)
  s_batch, a_batch, r_batch, s_prime_batch, done_batch = torch.tensor(s_lst, dtype=torch.float), torch.tensor(a_lst), \


# of episode :20, avg score : 17.6
# of episode :40, avg score : 18.1
# of episode :60, avg score : 19.2
# of episode :80, avg score : 16.1
# of episode :100, avg score : 20.4
# of episode :120, avg score : 17.2
# of episode :140, avg score : 18.6
# of episode :160, avg score : 17.8
# of episode :180, avg score : 17.3
# of episode :200, avg score : 16.9
# of episode :220, avg score : 19.6
# of episode :240, avg score : 18.6
# of episode :260, avg score : 23.0
# of episode :280, avg score : 23.8
# of episode :300, avg score : 21.4
# of episode :320, avg score : 28.4
# of episode :340, avg score : 34.0
# of episode :360, avg score : 33.0
# of episode :380, avg score : 32.3
# of episode :400, avg score : 33.6
# of episode :420, avg score : 36.0
# of episode :440, avg score : 37.2
# of episode :460, avg score : 33.5
# of episode :480, avg score : 34.0
# of episode :500, avg score : 36.4
# of episode :520, avg score : 56.1
# of episode :540, avg score : 52.1
# of episode :560, avg score : 6